In [32]:
import pandas as pd
import numpy as np
import seaborn as sns 
import matplotlib.pyplot as plt 

In [33]:
import requests
launch_data = requests.get("https://api.spacexdata.com/v4/launches").json()
data=pd.json_normalize(launch_data)


In [34]:
data.head()

,static_fire_date_utc,static_fire_date_unix,net,window,rocket,success,failures,details,crew,ships,...,links.reddit.media,links.reddit.recovery,links.flickr.small,links.flickr.original,links.presskit,links.webcast,links.youtube_id,links.article,links.wikipedia,fairings
0,2006-03-17T00:00:00.000Z,1.142554e+09,False,0.0,5e9d0d95eda69955f709d1eb,False,"[{'time': 33, 'altitude': None, 'reason': 'mer...",Engine failure at 33 seconds and loss of vehicle,[],[],...,None,None,[],[],None,https://www.youtube.com/watch?v=0a_00nJ_Y88,0a_00nJ_Y88,https://www.space.com/2196-spacex-inaugural-fa...,https://en.wikipedia.org/wiki/DemoSat,NaN
1,None,NaN,False,0.0,5e9d0d95eda69955f709d1eb,False,"[{'time': 301, 'altitude': 289, 'reason': 'har...",Successful first stage burn and transition to ...,[],[],...,None,None,[],[],None,https://www.youtube.com/watch?v=Lk4zQ2wP-Nc,Lk4zQ2wP-Nc,https://www.space.com/3590-spacex-falcon-1-roc...,https://en.wikipedia.org/wiki/DemoSat,NaN
2,None,NaN,False,0.0,5e9d0d95eda69955f709d1eb,False,"[{'time': 140, 'altitude': 35, 'reason': 'resi...",Residual stage 1 thrust led to collision betwe...,[],[],...,None,None,[],[],None,https://www.youtube.com/watch?v=v0w9p3U8860,v0w9p3U8860,http://www.spacex.com/news/2013/02/11/falcon-1...,https://en.wikipedia.org/wiki/Trailblazer_(sat...,NaN
3,2008-09-20T00:00:00.000Z,1.221869e+09,False,0.0,5e9d0d95eda69955f709d1eb,True,[],Ratsat was carried to orbit on the first succe...,[],[],...,None,None,[],[],None,https://www.youtube.com/watch?v=dLQ2tZEH6G0,dLQ2tZEH6G0,https://en.wikipedia.org/wiki/Ratsat,https://en.wikipedia.org/wiki/Ratsat,NaN
4,None,NaN,False,0.0,5e9d0d95eda69955f709d1eb,True,[],None,[],[],...,None,None,[],[],http://www.spacex.com/press/2012/12/19/spacexs...,https://www.youtube.com/watch?v=yTaIDooc8Og,yTaIDooc8Og,http://www.spacex.com/news/2013/02/12/falcon-1...,https://en.wikipedia.org/wiki/RazakSAT,NaN


In [35]:
data.shape

(205, 43)

## EDA

In [36]:
data.isna().sum()

static_fire_date_utc          84
static_fire_date_unix         84
net                            0
window                        88
rocket                         0
success                       19
failures                       0
details                       71
crew                           0
ships                          0
capsules                       0
payloads                       0
launchpad                      0
flight_number                  0
name                           0
date_utc                       0
date_unix                      0
date_local                     0
date_precision                 0
upcoming                       0
cores                          0
auto_update                    0
tbd                            0
launch_library_id            133
id                             0
fairings.reused              112
fairings.recovery_attempt     98
fairings.recovered           120
fairings.ships                36
links.patch.small             16
links.patc

In [37]:
data.columns

Index(['static_fire_date_utc', 'static_fire_date_unix', 'net', 'window',
       'rocket', 'success', 'failures', 'details', 'crew', 'ships', 'capsules',
       'payloads', 'launchpad', 'flight_number', 'name', 'date_utc',
       'date_unix', 'date_local', 'date_precision', 'upcoming', 'cores',
       'auto_update', 'tbd', 'launch_library_id', 'id', 'fairings.reused',
       'fairings.recovery_attempt', 'fairings.recovered', 'fairings.ships',
       'links.patch.small', 'links.patch.large', 'links.reddit.campaign',
       'links.reddit.launch', 'links.reddit.media', 'links.reddit.recovery',
       'links.flickr.small', 'links.flickr.original', 'links.presskit',
       'links.webcast', 'links.youtube_id', 'links.article', 'links.wikipedia',
       'fairings'],
      dtype='object')

### Data Cleaning  

i am taking these columns because they are usefulls and the reasons are montioned below , and i am droping rest of columns 


| Column                     | Why keep it?                                        |
| -------------------------- | --------------------------------------------------- |
| `success`                  | ✅ Target variable for prediction                    |
| `rocket`                   | ✅ Use to join with rocket type data                 |
| `launchpad`                | ✅ Use to join with launch site info                 |
| `payloads`                 | ✅ Use to fetch payload mass/type                    |
| `date_utc` or `date_local` | ✅ Date of launch (for trend analysis)               |
| `flight_number`            | ✅ Chronological order / trend                       |
| `name`                     | ✅ Launch name (for display)                         |
| `cores`                    | ✅ Can extract booster reuse, landing attempt, etc.  |
| `fairings.reused`          | ✅ Part of hardware reuse (optional but interesting) |


In [38]:
columns_to_drop = [
    'static_fire_date_utc', 'static_fire_date_unix', 'net', 'window',
    'failures', 'details', 'crew', 'ships', 'capsules',
    'date_unix', 'date_precision', 'upcoming', 'auto_update', 'tbd',
    'launch_library_id', 'id', 'fairings.ships',
    'links.patch.small', 'links.patch.large', 'links.reddit.campaign',
    'links.reddit.launch', 'links.reddit.media', 'links.reddit.recovery',
    'links.flickr.small', 'links.flickr.original', 'links.presskit',
    'links.webcast', 'links.youtube_id', 'links.article', 'links.wikipedia'
]


Reason for dropping these columns 

| Column                                          | Reason to drop                                      |
| ----------------------------------------------- | --------------------------------------------------- |
| `static_fire_date_utc`, `static_fire_date_unix` | Redundant and often missing                         |
| `net`, `window`                                 | Not informative for prediction                      |
| `failures`                                      | Redundant if `success` exists                       |
| `details`, `crew`                               | High text or nested complexity                      |
| `ships`, `capsules`                             | Irrelevant unless modeling logistics                |
| `date_unix`, `date_precision`                   | Redundant if using `date_utc`                       |
| `upcoming`                                      | Refers to future launches, not helpful for training |
| `auto_update`, `tbd`                            | Metadata, not predictive                            |
| `launch_library_id`, `id`                       | Internal IDs, not useful                            |
| `links.*`                                       | Media links, not useful for modeling                |
| `fairings.ships`                                | Unused and redundant                                |
| `links.*`                                       | Drop all `links.` columns (media URLs)              |


In [39]:
cleaned_data = data.drop(columns=columns_to_drop  )

In [40]:
cleaned_data.head()

,rocket,success,payloads,launchpad,flight_number,name,date_utc,date_local,cores,fairings.reused,fairings.recovery_attempt,fairings.recovered,fairings
0,5e9d0d95eda69955f709d1eb,False,[5eb0e4b5b6c3bb0006eeb1e1],5e9e4502f5090995de566f86,1,FalconSat,2006-03-24T22:30:00.000Z,2006-03-25T10:30:00+12:00,"[{'core': '5e9e289df35918033d3b2623', 'flight'...",False,False,False,NaN
1,5e9d0d95eda69955f709d1eb,False,[5eb0e4b6b6c3bb0006eeb1e2],5e9e4502f5090995de566f86,2,DemoSat,2007-03-21T01:10:00.000Z,2007-03-21T13:10:00+12:00,"[{'core': '5e9e289ef35918416a3b2624', 'flight'...",False,False,False,NaN
2,5e9d0d95eda69955f709d1eb,False,"[5eb0e4b6b6c3bb0006eeb1e3, 5eb0e4b6b6c3bb0006e...",5e9e4502f5090995de566f86,3,Trailblazer,2008-08-03T03:34:00.000Z,2008-08-03T15:34:00+12:00,"[{'core': '5e9e289ef3591814873b2625', 'flight'...",False,False,False,NaN
3,5e9d0d95eda69955f709d1eb,True,[5eb0e4b7b6c3bb0006eeb1e5],5e9e4502f5090995de566f86,4,RatSat,2008-09-28T23:15:00.000Z,2008-09-28T11:15:00+12:00,"[{'core': '5e9e289ef3591855dc3b2626', 'flight'...",False,False,False,NaN
4,5e9d0d95eda69955f709d1eb,True,[5eb0e4b7b6c3bb0006eeb1e6],5e9e4502f5090995de566f86,5,RazakSat,2009-07-13T03:35:00.000Z,2009-07-13T15:35:00+12:00,"[{'core': '5e9e289ef359184f103b2627', 'flight'...",False,False,False,NaN


In [51]:
cleaned_data['date_utc'] = pd.to_datetime(cleaned_data['date_utc'])
cleaned_data['date_local'] = pd.to_datetime(cleaned_data['date_local'])



C:\Users\ZA IT Park\AppData\Local\Temp\ipykernel_1480\2493783312.py:2: FutureWarning: In a future version of pandas, parsing datetimes with mixed time zones will raise an error unless `utc=True`. Please specify `utc=True` to opt in to the new behaviour and silence this warning. To create a `Series` with mixed offsets and `object` dtype, please use `apply` and `datetime.datetime.strptime`
  cleaned_data['date_local'] = pd.to_datetime(cleaned_data['date_local'])


In [53]:
cleaned_data.head(3)

,rocket,success,payloads,launchpad,flight_number,name,date_utc,date_local,cores,fairings.reused,fairings.recovery_attempt,fairings.recovered
0,5e9d0d95eda69955f709d1eb,False,[5eb0e4b5b6c3bb0006eeb1e1],5e9e4502f5090995de566f86,1,FalconSat,2006-03-24 22:30:00+00:00,2006-03-25 10:30:00+12:00,"[{'core': '5e9e289df35918033d3b2623', 'flight'...",False,False,False
1,5e9d0d95eda69955f709d1eb,False,[5eb0e4b6b6c3bb0006eeb1e2],5e9e4502f5090995de566f86,2,DemoSat,2007-03-21 01:10:00+00:00,2007-03-21 13:10:00+12:00,"[{'core': '5e9e289ef35918416a3b2624', 'flight'...",False,False,False
2,5e9d0d95eda69955f709d1eb,False,"[5eb0e4b6b6c3bb0006eeb1e3, 5eb0e4b6b6c3bb0006e...",5e9e4502f5090995de566f86,3,Trailblazer,2008-08-03 03:34:00+00:00,2008-08-03 15:34:00+12:00,"[{'core': '5e9e289ef3591814873b2625', 'flight'...",False,False,False


In [54]:
cleaned_data['cores'][1]

[{'core': '5e9e289ef35918416a3b2624',
  'flight': 1,
  'gridfins': False,
  'legs': False,
  'reused': False,
  'landing_attempt': False,
  'landing_success': None,
  'landing_type': None,
  'landpad': None}]

### extracting cores column

In [57]:
def extract_core_info(core_list):
    if core_list and isinstance(core_list, list) and core_list[0]:
        core = core_list[0]
        return pd.Series({
            'core_reused': core.get('reused'),
            'landing_attempt': core.get('landing_attempt'),
            'landing_success': core.get('landing_success'),
            'landing_type': core.get('landing_type')
        })
    else:
        return pd.Series({
            'core_reused': None,
            'landing_attempt': None,
            'landing_success': None,
            'landing_type': None
        })


In [59]:
# Apply the function to your DataFrame
core_info = cleaned_data['cores'].apply(extract_core_info)
cleaned_data = pd.concat([cleaned_data, core_info], axis=1)

In [62]:
cleaned_data.head(10)

,rocket,success,payloads,launchpad,flight_number,name,date_utc,date_local,fairings.reused,fairings.recovery_attempt,fairings.recovered,core_reused,landing_attempt,landing_success,landing_type
0,5e9d0d95eda69955f709d1eb,False,[5eb0e4b5b6c3bb0006eeb1e1],5e9e4502f5090995de566f86,1,FalconSat,2006-03-24 22:30:00+00:00,2006-03-25 10:30:00+12:00,False,False,False,False,False,None,None
1,5e9d0d95eda69955f709d1eb,False,[5eb0e4b6b6c3bb0006eeb1e2],5e9e4502f5090995de566f86,2,DemoSat,2007-03-21 01:10:00+00:00,2007-03-21 13:10:00+12:00,False,False,False,False,False,None,None
2,5e9d0d95eda69955f709d1eb,False,"[5eb0e4b6b6c3bb0006eeb1e3, 5eb0e4b6b6c3bb0006e...",5e9e4502f5090995de566f86,3,Trailblazer,2008-08-03 03:34:00+00:00,2008-08-03 15:34:00+12:00,False,False,False,False,False,None,None
3,5e9d0d95eda69955f709d1eb,True,[5eb0e4b7b6c3bb0006eeb1e5],5e9e4502f5090995de566f86,4,RatSat,2008-09-28 23:15:00+00:00,2008-09-28 11:15:00+12:00,False,False,False,False,False,None,None
4,5e9d0d95eda69955f709d1eb,True,[5eb0e4b7b6c3bb0006eeb1e6],5e9e4502f5090995de566f86,5,RazakSat,2009-07-13 03:35:00+00:00,2009-07-13 15:35:00+12:00,False,False,False,False,False,None,None
5,5e9d0d95eda69973a809d1ec,True,[5eb0e4b7b6c3bb0006eeb1e7],5e9e4501f509094ba4566f84,6,Falcon 9 Test Flight,2010-06-04 18:45:00+00:00,2010-06-04 14:45:00-04:00,unknown,unknown,unknown,False,False,None,None
6,5e9d0d95eda69973a809d1ec,True,"[5eb0e4b9b6c3bb0006eeb1e8, 5eb0e4b9b6c3bb0006e...",5e9e4501f509094ba4566f84,7,COTS 1,2010-12-08 15:43:00+00:00,2010-12-08 11:43:00-04:00,unknown,unknown,unknown,False,False,None,None
7,5e9d0d95eda69973a809d1ec,True,[5eb0e4bab6c3bb0006eeb1ea],5e9e4501f509094ba4566f84,8,COTS 2,2012-05-22 07:44:00+00:00,2012-05-22 03:44:00-04:00,unknown,unknown,unknown,False,False,None,None
8,5e9d0d95eda69973a809d1ec,True,"[5eb0e4bab6c3bb0006eeb1eb, 5eb0e4bab6c3bb0006e...",5e9e4501f509094ba4566f84,9,CRS-1,2012-10-08 00:35:00+00:00,2012-10-08 20:35:00-04:00,unknown,unknown,unknown,False,False,None,None
9,5e9d0d95eda69973a809d1ec,True,[5eb0e4bbb6c3bb0006eeb1ed],5e9e4501f509094ba4566f84,10,CRS-2,2013-03-01 19:10:00+00:00,2013-03-01 15:10:00-04:00,unknown,unknown,unknown,False,False,None,None


In [61]:
cleaned_data.drop(columns=['cores'], inplace=True)


In [52]:
cleaned_data['success'].value_counts()

success
True       181
unknown     19
False        5
Name: count, dtype: int64

In [63]:
cleaned_data.isnull().sum()

rocket                        0
success                       0
payloads                      0
launchpad                     0
flight_number                 0
name                          0
date_utc                      0
date_local                    0
fairings.reused               0
fairings.recovery_attempt     0
fairings.recovered            0
core_reused                   6
landing_attempt              18
landing_success              49
landing_type                 47
dtype: int64

In [43]:
cleaned_data['fairings.recovery_attempt'].unique()

array([False, None, nan, True], dtype=object)

In [44]:
del cleaned_data['fairings']

In [45]:
cleaned_data.columns

Index(['rocket', 'success', 'payloads', 'launchpad', 'flight_number', 'name',
       'date_utc', 'date_local', 'cores', 'fairings.reused',
       'fairings.recovery_attempt', 'fairings.recovered'],
      dtype='object')

In [64]:
cleaned_data.replace(np.NaN, 'unknown', inplace=True)

In [65]:
cleaned_data.isnull().sum()

rocket                       0
success                      0
payloads                     0
launchpad                    0
flight_number                0
name                         0
date_utc                     0
date_local                   0
fairings.reused              0
fairings.recovery_attempt    0
fairings.recovered           0
core_reused                  0
landing_attempt              0
landing_success              0
landing_type                 0
dtype: int64

In [66]:
cleaned_data.to_csv('cleaned_data_spacex_h.csv' ,index=False)

In [67]:
cleaned_data

,rocket,success,payloads,launchpad,flight_number,name,date_utc,date_local,fairings.reused,fairings.recovery_attempt,fairings.recovered,core_reused,landing_attempt,landing_success,landing_type
0,5e9d0d95eda69955f709d1eb,False,[5eb0e4b5b6c3bb0006eeb1e1],5e9e4502f5090995de566f86,1,FalconSat,2006-03-24 22:30:00+00:00,2006-03-25 10:30:00+12:00,False,False,False,False,False,unknown,unknown
1,5e9d0d95eda69955f709d1eb,False,[5eb0e4b6b6c3bb0006eeb1e2],5e9e4502f5090995de566f86,2,DemoSat,2007-03-21 01:10:00+00:00,2007-03-21 13:10:00+12:00,False,False,False,False,False,unknown,unknown
2,5e9d0d95eda69955f709d1eb,False,"[5eb0e4b6b6c3bb0006eeb1e3, 5eb0e4b6b6c3bb0006e...",5e9e4502f5090995de566f86,3,Trailblazer,2008-08-03 03:34:00+00:00,2008-08-03 15:34:00+12:00,False,False,False,False,False,unknown,unknown
3,5e9d0d95eda69955f709d1eb,True,[5eb0e4b7b6c3bb0006eeb1e5],5e9e4502f5090995de566f86,4,RatSat,2008-09-28 23:15:00+00:00,2008-09-28 11:15:00+12:00,False,False,False,False,False,unknown,unknown
4,5e9d0d95eda69955f709d1eb,True,[5eb0e4b7b6c3bb0006eeb1e6],5e9e4502f5090995de566f86,5,RazakSat,2009-07-13 03:35:00+00:00,2009-07-13 15:35:00+12:00,False,False,False,False,False,unknown,unknown
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
200,5e9d0d95eda69973a809d1ec,unknown,[],5e9e4501f509094ba4566f84,198,Transporter-6,2022-12-01 00:00:00+00:00,2022-11-30 19:00:00-05:00,unknown,unknown,unknown,False,unknown,unknown,unknown
201,5e9d0d95eda69973a809d1ec,unknown,[63161760ffc78f3b8567071c],5e9e4502f509092b78566f87,199,TTL-1,2022-12-01 00:00:00+00:00,2022-11-30 16:00:00-08:00,unknown,unknown,unknown,unknown,unknown,unknown,unknown
202,5e9d0d95eda69973a809d1ec,unknown,[6243b896af52800c6e91926e],5e9e4501f509094ba4566f84,200,WorldView Legion 1 & 2,2022-12-01 00:00:00+00:00,2022-11-30 19:00:00-05:00,unknown,unknown,unknown,unknown,unknown,unknown,unknown
203,5e9d0d95eda69974db09d1ed,unknown,[],5e9e4502f509094188566f88,202,Viasat-3 & Arcturus,2022-12-01 00:00:00+00:00,2022-11-30 19:00:00-05:00,unknown,unknown,unknown,False,unknown,unknown,unknown
